In [13]:
import pandas as pd
from datetime import datetime
transactions =[]
ALLOWED_FREQUENCIES = ("daily", "weekly", "monthly", "yearly") 

In [14]:
class Transaction:
    def __init__(self, category, amount, date=None):
        self.category = category
        self.amount = amount
        self.date = date or datetime.now().strftime("%Y-%m-%d")

class RecurringTransaction(Transaction):
    def __init__(self, category, amount, frequency, date=None):
        super().__init__(category, amount, date)
        self.frequency = frequency


In [15]:
#Transaction Functions
def validate_amount():
    while True:
        try:
            amt = float(input("Enter amount: "))
            if amt <= 0:
                raise ValueError("Amount must be positive.")
            return amt
        except ValueError as e:
            print("Error:", e)

def add_income():
    n = int(input("Enter how many type of incomes would you like to add : "))
    for _ in range(n) :
        amount=validate_amount()
        category= input("Enter type of Income:")
        date=input("Enter the date (YYYY-MM-DD) else press enter to select today")
        if date:
            try:
                datetime.striptime(date,"%Y-%m-%d")
            except ValueError:
                print("Invalid date format. using Today's date.")
                date=None
        else:
            date=None
        is_recurring=input("is this a recurring transaction(yes/no):").lower()
        if is_recurring == "yes":
            frequency=input("Enter frequency (daily/weekly/monthly/yearly):").lower()
            txn = RecurringTransaction(category, amount, frequency, date)
        else:
            txn = Transaction(category, amount, date)
        transactions.append(txn)
        print("Income added successfully.")

      
def add_expense():
    n = int(input("Enter how many type of expenses would you like to add : "))
    for _ in range(n) :
        expense=validate_amount()
        category= input("Enter type of expense:")
        date=input("Enter the date (YYYY-MM-DD) else press enter to select today")
        if date:
            try:
                datetime.striptime(date,"%Y-%m-%d")
            except ValueError:
                print("Invalid date format. using Today's date.")
                date=None
        else:
            date=None
        is_recurring=input("is this a recurring transaction(yes/no):").lower()
        if is_recurring == "yes":
            frequency=input("Enter frequency (daily/weekly/monthly/yearly):").lower()
            txn = RecurringTransaction(category, -expense, frequency, date)
        else:
            txn = Transaction(category,-expense, date)
        transactions.append(txn)
        print("expense added successfully.")


In [16]:

#summary functions

def display_transactions():
    if not transactions:
        print("No record of transaction found")
        return
    for t in transactions:
        ttype="Income" if t.amount>0 else "Expense"
        details= f"{ttype}-{t.category}-{abs(t.amount)}-{t.date}"
        if isinstance(t,RecurringTransaction):
            details += f"-Frequency:{t.frequency}"
        print(details)

def calculate_savings():
    total_income=sum(t.amount for t in transactions if t.amount>0)
    total_expense=-sum(t.amount for t in transactions if t.amount<0)
    print("Total Savings:", total_income-total_expense)
    return total_income, total_expense

def view_summary():
    total_income, total_expense = calculate_savings()
    print("total income is :", total_income)
    print("total expense is :", total_expense)
    print("total savings is :", (total_income-total_expense))


In [17]:
def search_transactions():
    keyword=input("enter your search category: ").lower()
    found = False 
    for t in transactions:
        if keyword in t.category.lower():
            print(f"{t.category} - {abs(t.amount)} - {t.date}", end="")

            if isinstance(t, RecurringTransaction):
                print(f" - Frequency: {t.frequency}")
            else:
                print() 
            found = True
    if not found:
        print("No matching transactions found.")
    
def get_highest_spending_category():
    total = {}
    for t in transactions:
        if t.amount<0:
            if t.category in total:
                total[t.category]+= -t.amount
            else:
                total[t.category]=-t.amount
    if not total:
        return None,0
    
    highest_cat=max(total, key=total.get)
    highest_amount=total[highest_cat]
    return highest_cat, highest_amount


def category_insights():
   
    cat,amount= get_highest_spending_category()
    if cat:
        print("Highest spending category:",cat)
        print("Amount Spent:", amount)
    else:
        print("No recorded expenses found")

def delete_transactions():
    d=input("Enter the category to erase: ").lower()
    found= False
    for t in transactions:
        if t.category.lower() == d:
            transactions.remove(t)
            print("Transaction is deleted")
            found=True
            break
    if not found:
        print("No records found")

In [ ]:
def export_data():
    try:
        data = []
        for t in transactions:
            row = {
                "Category": t.category,
                "Amount": t.amount,
                "Date": t.date,
                "Type": "Recurring" if isinstance(t, RecurringTransaction) else "One-time",
                "Frequency": t.frequency if isinstance(t, RecurringTransaction) else ""
            }
            data.append(row)
        df = pd.DataFrame(data)
        df.to_csv("my_transactions_export.csv", index=False)
        print("Data exported successfully to 'my_transactions_export.csv'")
    except Exception as e:
        print("Error exporting data:", e)

def import_data():
    try:
        df = pd.read_csv("my_transactions_export.csv")  # ✅ Match export filename
        transactions.clear()

        for _, row in df.iterrows():
            category = row["Category"]
            amount = float(row["Amount"])
            date = row["Date"]
            frequency = str(row.get("Frequency", "") or "").strip()
            type_info = str(row.get("Type", "") or "").strip()

            if type_info.lower() == "recurring" and frequency:
                txn = RecurringTransaction(category, amount, frequency, date)
            else:
                txn = Transaction(category, amount, date)
            transactions.append(txn)

        print("Data imported successfully!!")

    except FileNotFoundError:
        print("No saved data found. Starting fresh.")

    except Exception as e:
        print("Error importing data:", e)

    finally:
        print("import_data()-function execution completed.\n")


In [19]:
def main_menu():
    while True:
        try:
            print("\n===== Personal Finance Tracker =====")
            print("1. Add Income")
            print("2. Add Expense")
            print("3. View Summary")
            print("4. Search Transactions")
            print("5. Category Insights")
            print("6. Delete Transactions")
            print("7. Display All Transactions")
            print("8. Export Data")
            print("9. Import Data")
            print("10. Exit")

            choice = input("Choose an option: ").strip()

            if choice == "1":
                add_income()
            elif choice == "2":
                add_expense()
            elif choice == "3":
                view_summary()
            elif choice == "4":
                search_transactions()
            elif choice == "5":
                category_insights()
            elif choice == "6":
                delete_transactions()
            elif choice == "7":
                display_transactions()
            elif choice == "8":
                export_data()
            elif choice == "9":
                import_data()
            elif choice == "10":
                print("Exiting program... Goodbye!")
                break
            else:
                print("Invalid choice. Please try again.")

        except ValueError:
            print("Invalid input! Please enter a valid number or option.")
        except IndexError:
            print("Index error: you might be trying to access data that doesn't exist yet.")
        except Exception as e:
            print(f"Unexpected error occurred: {e}")

In [20]:
main_menu()


===== Personal Finance Tracker =====
1. Add Income
2. Add Expense
3. View Summary
4. Search Transactions
5. Category Insights
6. Delete Transactions
7. Display All Transactions
8. Export Data
9. Import Data
10. Exit
Income added successfully.
Income added successfully.

===== Personal Finance Tracker =====
1. Add Income
2. Add Expense
3. View Summary
4. Search Transactions
5. Category Insights
6. Delete Transactions
7. Display All Transactions
8. Export Data
9. Import Data
10. Exit
expense added successfully.
expense added successfully.

===== Personal Finance Tracker =====
1. Add Income
2. Add Expense
3. View Summary
4. Search Transactions
5. Category Insights
6. Delete Transactions
7. Display All Transactions
8. Export Data
9. Import Data
10. Exit
Total Savings: 443.0
total income is : 952.0
total expense is : 509.0
total savings is : 443.0

===== Personal Finance Tracker =====
1. Add Income
2. Add Expense
3. View Summary
4. Search Transactions
5. Category Insights
6. Delete Transac